In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB
import highspy

from utils.timer import Timer

from numba import njit, jit

from utils.constraintUtils import *
from utils.formattingUtils import *
from utils.constructILPUtils import *
from utils.solveILPUtils import *


In [2]:
#The sudoku from the problem
empty_sudoku = np.array(
   [[0, 0, 1, 0, 0, 0, 0, 0, 2],
    [0, 0, 0, 3, 0, 4, 5, 0, 0],
    [6, 0, 0, 7, 0, 0, 1, 0, 0],
    [0, 4, 0, 5, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 8, 0],
    [0, 0, 0, 0, 0, 6, 0, 9, 0],
    [0, 0, 5, 0, 0, 9, 0, 0, 4],
    [0, 0, 8, 2, 0, 1, 0, 0, 0],
    [3, 0, 0, 0, 0, 0, 7, 0, 0]]
)

#a sample sudoku from the internet
sample_solution = np.array(
    [[7, 1, 3, 5, 2, 4, 6, 9, 8],
     [5, 2, 9, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#the same sample sudoku with some elements deleted
empty_sample_solution = np.array(
    [[0, 0, 0, 5, 2, 4, 6, 9, 8],
     [5, 2, 0, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#A very empty sudoku
very_empty_clues = np.zeros((9,9), dtype = int)
very_empty_clues[0,1] = 3
very_empty_clues[0,2] = 2
very_empty_clues[0,3] = 1

In [3]:
bin_sample_solution = array_to_vector(sample_solution)
vector_to_array(bin_sample_solution)

array([[7, 1, 3, 5, 2, 4, 6, 9, 8],
       [5, 2, 9, 6, 1, 8, 3, 4, 7],
       [6, 4, 8, 7, 3, 9, 2, 5, 1],
       [1, 5, 2, 9, 4, 7, 8, 3, 6],
       [8, 3, 6, 1, 5, 2, 9, 7, 4],
       [4, 9, 7, 3, 8, 6, 5, 1, 2],
       [3, 8, 5, 4, 6, 1, 7, 2, 9],
       [9, 6, 1, 2, 7, 5, 4, 8, 3],
       [2, 7, 4, 8, 9, 3, 1, 6, 5]])

In [4]:
for i in range(10):

    with Timer("Construnct ILP"):
        c, A_full_sparse = constuct_A_sparse_and_c(empty_sudoku)
        
    with Timer("Solve With Scipy"):
        solve_sudoku_scipy(c, A_full_sparse)

In [5]:
print(Timer.timers)

{'Construnct ILP': [3.1665288121439517, 0.0010892681311815977, 0.0007735551334917545, 0.000761949922889471, 0.0007653620559722185, 0.0007603189442306757, 0.0007642391137778759, 0.0008443729020655155, 0.0008527080062776804, 0.0007761409506201744], 'Solve With Scipy': [0.009723775088787079, 0.004479357041418552, 0.0034458341542631388, 0.0034331800416111946, 0.0034676450304687023, 0.0034335670061409473, 0.0034871441312134266, 0.0038090210873633623, 0.003680960973724723, 0.0034336689859628677]}


In [6]:
import gurobipy as gp
from gurobipy import GRB
import highspy

env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()
		
def solve_sudoku_gurobi(c, A_full):
	 # Create a new model
	 m = gp.Model("matrix1", env = env) 
	 # Create variables
	 solution_size = A_full.shape[1]
	 x = m.addMVar(shape=solution_size, vtype=GRB.BINARY, name="x")
	 # Build rhs vector
	 rhs = np.ones(A_full.shape[0], dtype = int)
	 # Add constraints
	 m.addConstr(A_full @ x == rhs, name="constraints")
	 # Set objective
	 m.setObjective(c @ x, GRB.MINIMIZE)
	 # Optimize model
	 m.optimize()
	 return x.X
	

def solve_sudoku_highspy(c, A_full):

	 rows, cols = A_full.shape
	 # Highs h
	 h = highspy.Highs()

	 # Define a HighsLp instance
	 lp = highspy.HighsLp()
	
	 lp.num_col_ = cols
	 lp.num_row_ = rows
	 lp.col_cost_ = c
	 lp.col_lower_ = np.zeros(c.shape, dtype = int)
	 lp.col_upper_ = np.ones(c.shape, dtype = int)
	 lp.row_lower_ = np.ones((rows), dtype = int)
	 lp.row_upper_ = np.ones((rows), dtype = int)
	 lp.integrality_ = [highspy._core.HighsVarType.kInteger]*cols # np.ones(c.shape, dtype = int).tolist()
	 #lp.integrality_ = 1#np.ones(c.shape, dtype = int)#highspy.HighsVarType.kInteger
	
	 # In a HighsLp instsance, the number of nonzeros is given by a fictitious final start
	 lp.a_matrix_.format_ = highspy.MatrixFormat.kColwise
	 lp.a_matrix_.start_ = A_full.indptr
	 lp.a_matrix_.index_ = A_full.indices
	 lp.a_matrix_.value_ = A_full.data
	 h.passModel(lp)

	 # Get and set options
	 options = h.getOptions()
	 options.log_to_console = False
	 h.passOptions(options)

	 h.run()
	
	 solution = h.getSolution()
	 x= np.array(solution.col_value)
	 return x


In [7]:
with Timer("njitted"):
    setup_ILP_njit(empty_sudoku)

with Timer("normal"):
    setup_ILP_dense(empty_sudoku)

print(Timer.timers)

{'Construnct ILP': [3.1665288121439517, 0.0010892681311815977, 0.0007735551334917545, 0.000761949922889471, 0.0007653620559722185, 0.0007603189442306757, 0.0007642391137778759, 0.0008443729020655155, 0.0008527080062776804, 0.0007761409506201744], 'Solve With Scipy': [0.009723775088787079, 0.004479357041418552, 0.0034458341542631388, 0.0034331800416111946, 0.0034676450304687023, 0.0034335670061409473, 0.0034871441312134266, 0.0038090210873633623, 0.003680960973724723, 0.0034336689859628677], 'njitted': [2.31788611295633], 'normal': [0.002248200122267008]}


In [8]:
with Timer("Solve Highs"):
    c, A_full_njit = setup_ILP_njit(empty_sudoku)
    A_full_njit_sparse = sp.sparse.csc_matrix(A_full_njit)
    #model = solve_sudoku_pisciopt(c, A_full_njit)
    solution = solve_sudoku_highspy(c, A_full_njit_sparse)

print(vector_to_array(solution))
print(Timer.timers)

Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
[[7 3 1 8 6 5 9 4 2]
 [8 9 2 3 1 4 5 6 7]
 [6 5 4 7 9 2 1 3 8]
 [9 4 6 5 8 3 2 7 1]
 [1 2 3 9 4 7 6 8 5]
 [5 8 7 1 2 6 4 9 3]
 [2 7 5 6 3 9 8 1 4]
 [4 6 8 2 7 1 3 5 9]
 [3 1 9 4 5 8 7 2 6]]
{'Construnct ILP': [3.1665288121439517, 0.0010892681311815977, 0.0007735551334917545, 0.000761949922889471, 0.0007653620559722185, 0.0007603189442306757, 0.0007642391137778759, 0.0008443729020655155, 0.0008527080062776804, 0.0007761409506201744], 'Solve With Scipy': [0.009723775088787079, 0.004479357041418552, 0.0034458341542631388, 0.0034331800416111946, 0.0034676450304687023, 0.0034335670061409473, 0.0034871441312134266, 0.0038090210873633623, 0.003680960973724723, 0.0034336689859628677], 'njitted': [2.31788611295633], 'normal': [0.002248200122267008], 'Solve Highs': [0.016227076994255185]}


In [9]:
#fastest option
with Timer("Solve sudoku"):
    c, A_full_njit = setup_ILP_njit(empty_sudoku)
    A_sparse_njit =  sp.sparse.csr_matrix(A_full_njit)
    res_scipy = solve_sudoku_scipy(c, A_sparse_njit)
    

print(Timer.timers)
    
    

{'Construnct ILP': [3.1665288121439517, 0.0010892681311815977, 0.0007735551334917545, 0.000761949922889471, 0.0007653620559722185, 0.0007603189442306757, 0.0007642391137778759, 0.0008443729020655155, 0.0008527080062776804, 0.0007761409506201744], 'Solve With Scipy': [0.009723775088787079, 0.004479357041418552, 0.0034458341542631388, 0.0034331800416111946, 0.0034676450304687023, 0.0034335670061409473, 0.0034871441312134266, 0.0038090210873633623, 0.003680960973724723, 0.0034336689859628677], 'njitted': [2.31788611295633], 'normal': [0.002248200122267008], 'Solve Highs': [0.016227076994255185], 'Solve sudoku': [0.005434246966615319]}


In [10]:
#fastest option

c, A_full_njit = setup_ILP_njit(empty_sudoku)

for i in range(10):
    A_sparse_njit =  sp.sparse.csr_matrix(A_full_njit)
    with Timer("CSR"):
        solve_sudoku_scipy(c, A_sparse_njit)
    
    A_sparse_njit =  sp.sparse.csc_matrix(A_full_njit)
    with Timer("CSC"):
        solve_sudoku_scipy(c, A_sparse_njit)
    

print(Timer.timers)
    

{'Construnct ILP': [3.1665288121439517, 0.0010892681311815977, 0.0007735551334917545, 0.000761949922889471, 0.0007653620559722185, 0.0007603189442306757, 0.0007642391137778759, 0.0008443729020655155, 0.0008527080062776804, 0.0007761409506201744], 'Solve With Scipy': [0.009723775088787079, 0.004479357041418552, 0.0034458341542631388, 0.0034331800416111946, 0.0034676450304687023, 0.0034335670061409473, 0.0034871441312134266, 0.0038090210873633623, 0.003680960973724723, 0.0034336689859628677], 'njitted': [2.31788611295633], 'normal': [0.002248200122267008], 'Solve Highs': [0.016227076994255185], 'Solve sudoku': [0.005434246966615319], 'CSR': [0.003492502961307764, 0.004855858162045479, 0.006407992914319038, 0.006423999089747667, 0.00642227497883141, 0.006438957992941141, 0.006436220835894346, 0.006421050988137722, 0.006505716126412153, 0.00644761580042541], 'CSC': [0.003369739977642894, 0.006307727890089154, 0.006283740978688002, 0.0062801779713481665, 0.006291904952377081, 0.006311337929

In [11]:
repeats = 8
for i in range(repeats):
    with Timer("1. Setup njit dense"):
        c, A_full_njit_dense = setup_ILP_njit(empty_sudoku)
    with Timer("2. Scipy solver njit dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_njit_dense)
    with Timer("3. Gurobi solver njit dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_njit_dense )
    del c
    del A_full_njit_dense 

    with Timer("1. Setup njit sparse"):
        c, A_full_njit_dense = setup_ILP_njit(empty_sudoku)
        A_full_njit_sparse = sp.sparse.csc_matrix(A_full_njit_dense)
    with Timer("2. Scipy solver njit sparse"):
        res_scipy = solve_sudoku_scipy(c, A_full_njit_sparse )
    with Timer("3. Gurobi solver njit sparse"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_njit_sparse )
    with Timer("4. Highspy solver njit sparse"):
        res_highspy = solve_sudoku_highspy(c, A_full_njit_sparse )
    del c
    del A_full_njit_sparse
        
    with Timer("1. Setup sparse_dense"):
        c, A_full_sparse = setup_ILP_sparse_end(empty_sudoku, format = "csc", dtype = bool)
    with Timer("2. Scipy solver sparse_dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_sparse)
    with Timer("3. Gurobi solver sparse_dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_sparse)
    del c
    del A_full_sparse
    
    with Timer("1. Setup sparse"):
        c, A_full_sparse = setup_ILP_sparse(empty_sudoku, format = "csc", dtype = bool)
    with Timer("2. Scipy solver sparse"):
        res_scipy = solve_sudoku_scipy(c, A_full_sparse)
    with Timer("3. Gurobi solver sparse"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_sparse)
    del c
    del A_full_sparse
    
    with Timer("1. Setup dense"):
        c, A_full_dense = setup_ILP_dense(empty_sudoku, dtype = bool)
    with Timer("2. Scipy solver dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_dense)
    with Timer("3. Gurobi solver dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_dense)
    del c
    del A_full_dense

np.set_printoptions(precision=5)
keys = np.sort(list(Timer.timers.keys()))
for key in keys:
    print(key, "\t\t", np.array(Timer.timers[key]))

Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
Running HiGHS 1.7.1 (git hash: 0c240d8): Copyright (c) 2024 HiGHS under MIT licence terms
1. Setup dense 		 [0.0007  0.00069 0.00068 0.00069 0.0007  0.00069 0.00072 0.00071]
1. Setup njit dense 		 [0.00066 0.00064 0.00064 0.00064 0.00065 0.00065 0.00065 0.0008 ]
1. Setup njit sparse 		 [0.00194 0.00192 0.00194 0.00195 0.00201 0.00199 0.00197 0.00198]
1. Setup sparse 	